In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import random
from torch.utils.data import Dataset, DataLoader


In [ ]:
# 1️⃣ Define Vocabulary
vocab = ["I", "like", "maths", "neural", "networks", "are", "great", "deep", "learning", "fun"]
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Special tokens
START_TOKEN = len(vocab)  # Index for <START>
STOP_TOKEN = len(vocab) + 1  # Index for <STOP>

# 2️⃣ Generate Variable-Length Sequences
def generate_random_sequences(n_samples=1000, max_len=6):
    sequences = []
    for _ in range(n_samples):
        length = random.randint(2, max_len)
        seq = random.choices(vocab, k=length)
        sequences.append(seq)
    return sequences

random_sequences = generate_random_sequences()
df = pd.DataFrame({"sentence": random_sequences})

# 3️⃣ Create Targets for Next-Word Prediction
def create_next_word_targets(sentence):
    encoded = [word_to_idx[word] for word in sentence]
    input_seq = [START_TOKEN] + encoded  # Add <START>
    target_seq = encoded + [STOP_TOKEN]  # Add <STOP>
    return input_seq, target_seq

df["next_word_x"], df["next_word_y"] = zip(*df["sentence"].apply(create_next_word_targets))

# Convert to tensors
X_next_word = torch.tensor(df["next_word_x"].tolist(), dtype=torch.long)
Y_next_word = torch.tensor(df["next_word_y"].tolist(), dtype=torch.long)

# 4️⃣ Create Targets for Sentiment Analysis
def generate_fake_sentiment():
    return random.choice([0, 1])

df["sentiment_y"] = df["sentence"].apply(lambda x: generate_fake_sentiment())
Y_sentiment = torch.tensor(df["sentiment_y"].tolist(), dtype=torch.long)

# 5️⃣ Define Dataset & DataLoader
class TextDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

dataset_next_word = TextDataset(X_next_word, Y_next_word)
dataset_sentiment = TextDataset(X_next_word, Y_sentiment)
dataloader_next_word = DataLoader(dataset_next_word, batch_size=16, shuffle=True)
dataloader_sentiment = DataLoader(dataset_sentiment, batch_size=16, shuffle=True)

# 6️⃣ Define RNN, LSTM, GRU Classes
class BaseModel(nn.Module):
    def __init__(self, model_type, vocab_size, embed_dim=16, hidden_dim=32, output_dim=10):
        super(BaseModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size + 2, embed_dim)  # +2 for special tokens
        if model_type == "RNN":
            self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        elif model_type == "LSTM":
            self.rnn = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        elif model_type == "GRU":
            self.rnn = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.embedding(x)
        _, hidden = self.rnn(x)
        if isinstance(hidden, tuple):  # LSTM case
            hidden = hidden[0]
        return self.fc(hidden.squeeze(0))

# Instantiate models
rnn_model = BaseModel("RNN", len(vocab))
lstm_model = BaseModel("LSTM", len(vocab))
gru_model = BaseModel("GRU", len(vocab))

# 7️⃣ Training Functions
def train_next_word(model, optimizer, dataloader, epochs=5):
    model.train()
    criterion = nn.CrossEntropyLoss(ignore_index=STOP_TOKEN)
    for epoch in range(epochs):
        total_loss = 0
        for batch_x, batch_y in dataloader:
            batch_x = batch_x[:, :-1]
            batch_y = batch_y[:, 1:]
            optimizer.zero_grad()
            output = model(batch_x)
            loss = criterion(output.view(-1, output.shape[-1]), batch_y.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

def train_sentiment(model, optimizer, dataloader, epochs=5):
    model.train()
    criterion = nn.BCEWithLogitsLoss()
    for epoch in range(epochs):
        total_loss = 0
        for batch_x, batch_y in dataloader:
            optimizer.zero_grad()
            output = model(batch_x).squeeze(1)
            loss = criterion(output.float(), batch_y.float())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# 8️⃣ Train the Models
optimizer_rnn = optim.Adam(rnn_model.parameters(), lr=0.01)
optimizer_lstm = optim.Adam(lstm_model.parameters(), lr=0.01)
optimizer_gru = optim.Adam(gru_model.parameters(), lr=0.01)

print("Training RNN for Next-Word Prediction...")
train_next_word(rnn_model, optimizer_rnn, dataloader_next_word)

print("Training LSTM for Next-Word Prediction...")
train_next_word(lstm_model, optimizer_lstm, dataloader_next_word)

print("Training GRU for Next-Word Prediction...")
train_next_word(gru_model, optimizer_gru, dataloader_next_word)

print("Training RNN for Sentiment Analysis...")
train_sentiment(rnn_model, optimizer_rnn, dataloader_sentiment)

print("Training LSTM for Sentiment Analysis...")
train_sentiment(lstm_model, optimizer_lstm, dataloader_sentiment)

print("Training GRU for Sentiment Analysis...")
train_sentiment(gru_model, optimizer_gru, dataloader_sentiment)